In [1]:
from graphdatascience import GraphDataScience

# Configures the driver with AuraDS-recommended settings
gds = GraphDataScience(
    "neo4j+ssc://35.181.44.209:7687",
    auth=("neo4j", "neo4jdss")
)

print(gds.version())

2.0.2


In [5]:
# Sets database (if not default one)
gds.set_database("wine")

In [0]:
# Create a minimal similarity graph
gds.run_cypher(
  """
  
  """
)